In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Função para calcular distâncias entre aeroportos
def achar_distancias(lista_nacionais, lista_interesse, df):
    df_filtrado = df[
        (df['InputID'].isin(lista_nacionais)) &
        (df['TargetID'].isin(lista_interesse))
    ]
    distancias = {}
    for nacional in lista_nacionais:
        distancias[nacional] = {}
        for interesse in lista_interesse:
            distancia = df_filtrado[
                (df_filtrado['InputID'] == nacional) & 
                (df_filtrado['TargetID'] == interesse)
            ]['Distância km']
            distancias[nacional][interesse] = distancia.values[0] if not distancia.empty else None
    return distancias

# Função para carregar o grafo a partir do DataFrame
def load_graph_from_df(df, source_airports, interest_airports, local_airport):
    distancia = achar_distancias(interest_airports, local_airport, df)
    filtered_df = df[
        (df['InputID'].isin(source_airports)) & 
        (df['TargetID'].isin(interest_airports))
    ]
    G = nx.DiGraph()
    for _, linha in filtered_df.iterrows():
        source = linha['InputID']
        target = linha['TargetID']
        weight = linha['Distância km']
        
        G.add_edge(source, target, weight=weight)
    for airport in interest_airports:
        for local in local_airport:
            if not G.has_edge(airport, local):
                weight = distancia.get(airport, {}).get(local, 0)
                G.add_edge(airport, local, weight=weight)
    return G

# Função para destacar as rotas ideais
def highlight_routes(G, international_airports, national_airports, regional_airports):
    ideal_path_edges = set()
    for intl_airport in international_airports:
        if intl_airport not in G:
            continue
        try:
            min_total_weight = float('inf')
            best_path_to_regional = None
            for regional_airport in regional_airports:
                if regional_airport not in G:
                    continue
                for national_airport in national_airports:
                    if national_airport not in G:
                        continue
                    try:
                        path_to_national = nx.shortest_path(G, source=intl_airport, target=national_airport, weight='weight')
                        weight_to_national = nx.path_weight(G, path_to_national, weight='weight')
                        path_to_regional = nx.shortest_path(G, source=national_airport, target=regional_airport, weight='weight')
                        weight_to_regional = nx.path_weight(G, path_to_regional, weight='weight')
                        total_weight = weight_to_national + weight_to_regional
                        if total_weight < min_total_weight:
                            min_total_weight = total_weight
                            best_path_to_regional = path_to_national + path_to_regional[1:]
                    except nx.NetworkXNoPath:
                        continue
            if best_path_to_regional:
                for i in range(len(best_path_to_regional) - 1):
                    edge = (best_path_to_regional[i], best_path_to_regional[i + 1])
                    ideal_path_edges.add(edge)
        except nx.NetworkXNoPath:
            continue
    return list(ideal_path_edges)

# Função para plotar o grafo
def plot_clean_graph(G, ideal_path_edges, interest_airports, local_airports, title_suffix):
    pos = nx.circular_layout(G)
    plt.figure(figsize=(12, 10))
    node_colors = [
        'orange' if node in local_airports else 'lightgreen' if node in interest_airports else 'lightblue'
        for node in G.nodes
    ]
    node_labels = {
        node: aeroportos_nome[node] for node in G.nodes
    }
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=800, alpha=0.9)
    regular_edges = [edge for edge in G.edges if edge not in ideal_path_edges]
    nx.draw_networkx_edges(G, pos, edgelist=regular_edges, edge_color='gray', alpha=0.5, width=1)
    nx.draw_networkx_edges(G, pos, edgelist=ideal_path_edges, edge_color='red', width=2.5)
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=9, font_color='black')
    edge_labels = nx.get_edge_attributes(G, 'weight')
    highlighted_labels = {edge: f"{weight}" for edge, weight in edge_labels.items() if edge in ideal_path_edges}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=highlighted_labels, font_size=8, font_color='red')
    plt.title(f'{title_suffix}')
    plt.axis('off')
    plt.show()

tabela_rotas = []

# Loop para processar aeroportos em lotes
for i in aeroportos_noHub.keys():
    batch_size = 10
    for j in range(0, len(rotas_originais[i][0]), batch_size):
        aeroportos_intermediarios = rotas_originais[i][0][j:j + batch_size]
        subset_origem = rotas_originais[i][1][j:j + batch_size]
        G = load_graph_from_df(df, subset_origem, aeroportos_intermediarios, aeroportos_noHub[i])
        ideal_path_edges = highlight_routes(G, subset_origem, aeroportos_intermediarios, aeroportos_noHub[i])        
        tabela_rotas = create_table(tabela_rotas,ideal_path_edges,subset_origem, aeroportos_noHub[i],i,df)
        plot_clean_graph(G, ideal_path_edges, aeroportos_intermediarios, aeroportos_noHub[i], title_suffix=f"{i}")

ValueError: range() arg 3 must not be zero

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Função para calcular distâncias entre aeroportos
def achar_distancias(lista_nacionais, lista_interesse, df):
    df_filtrado = df[
        (df['InputID'].isin(lista_nacionais)) &
        (df['TargetID'].isin(lista_interesse))
    ]
    distancias = {}
    for nacional in lista_nacionais:
        distancias[nacional] = {}
        for interesse in lista_interesse:
            distancia = df_filtrado[
                (df_filtrado['InputID'] == nacional) & 
                (df_filtrado['TargetID'] == interesse)
            ]['Distância km']
            distancias[nacional][interesse] = distancia.values[0] if not distancia.empty else None
    return distancias

# Função para carregar o grafo a partir do DataFrame
def load_graph_from_df(df, source_airports, interest_airports, local_airport):
    distancia = achar_distancias(interest_airports, local_airport, df)
    filtered_df = df[
        (df['InputID'].isin(source_airports)) & 
        (df['TargetID'].isin(interest_airports))
    ]
    G = nx.DiGraph()
    for _, linha in filtered_df.iterrows():
        source = linha['InputID']
        target = linha['TargetID']
        weight = linha['Distância km']
        if target not in local_airport:
            for i in local_airport:
                G.add_edge(source, target, weight=weight)
    for airport in interest_airports:
        for local in local_airport:
            if not G.has_edge(airport, local):
                weight = distancia.get(airport, {}).get(local, 0)
                G.add_edge(airport, local, weight=weight)
    return G

# Função para destacar as rotas ideais
def highlight_routes(G, international_airports, national_airports, regional_airports):
    ideal_path_edges = set()
    for intl_airport in international_airports:
        if intl_airport not in G:
            continue
        try:
            min_total_weight = float('inf')
            best_path_to_regional = None
            for regional_airport in regional_airports:
                if regional_airport not in G:
                    continue
                for national_airport in national_airports:
                    if national_airport not in G:
                        continue
                    try:
                        path_to_national = nx.shortest_path(G, source=intl_airport, target=national_airport, weight='weight')
                        weight_to_national = nx.path_weight(G, path_to_national, weight='weight')
                        path_to_regional = nx.shortest_path(G, source=national_airport, target=regional_airport, weight='weight')
                        weight_to_regional = nx.path_weight(G, path_to_regional, weight='weight')
                        total_weight = weight_to_national + weight_to_regional
                        if total_weight < min_total_weight:
                            min_total_weight = total_weight
                            best_path_to_regional = path_to_national + path_to_regional[1:]
                    except nx.NetworkXNoPath:
                        continue
            if best_path_to_regional:
                for i in range(len(best_path_to_regional) - 1):
                    edge = (best_path_to_regional[i], best_path_to_regional[i + 1])
                    ideal_path_edges.add(edge)
        except nx.NetworkXNoPath:
            continue
    return list(ideal_path_edges)

# Função para plotar o grafo
def plot_clean_graph(G, ideal_path_edges, interest_airports, local_airports, title_suffix):
    pos = nx.circular_layout(G)
    plt.figure(figsize=(12, 10))
    node_colors = [
        'orange' if node in local_airports else 'lightgreen' if node in interest_airports else 'lightblue'
        for node in G.nodes
    ]
    node_labels = {
        node: aeroportos_nome[node] for node in G.nodes
    }
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=800, alpha=0.9)
    regular_edges = [edge for edge in G.edges if edge not in ideal_path_edges]
    nx.draw_networkx_edges(G, pos, edgelist=regular_edges, edge_color='gray', alpha=0.5, width=1)
    nx.draw_networkx_edges(G, pos, edgelist=ideal_path_edges, edge_color='red', width=2.5)
    nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=9, font_color='black')
    edge_labels = nx.get_edge_attributes(G, 'weight')
    highlighted_labels = {edge: f"{weight}" for edge, weight in edge_labels.items() if edge in ideal_path_edges}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=highlighted_labels, font_size=8, font_color='red')
    plt.title(f'{title_suffix}')
    plt.axis('off')
    plt.show()

tabela_hub = []
# Loop para processar aeroportos em lotes
for i in aeroportos_noHub.keys():
    batch_size = 10
    for j in range(0, len(aeroportos_rotas[i]), batch_size):
        subset_origem = aeroportos_rotas[i][j:j + batch_size]
        G = load_graph_from_df(df, subset_origem, aeroportos_Hub, aeroportos_noHub[i])
        ideal_path_edges = highlight_routes(G, subset_origem, aeroportos_Hub, aeroportos_noHub[i])
        tabela_hub = create_table(tabela_hub,ideal_path_edges,subset_origem, aeroportos_noHub[i],i,df)
        plot_clean_graph(G, ideal_path_edges, aeroportos_Hub, aeroportos_noHub[i], title_suffix=f"{i}")